In [1]:
import multiprocessing
import os
import time
import numpy as np
from scipy.spatial import cKDTree
import open3d as o3d
import util
from tqdm import tqdm
import matplotlib.pyplot as plt
from matplotlib import cm
import random

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
kdtree = None  # Global k-d tree

def initKdtree(tree):
    """Initialize k-d tree in each worker process."""
    global kdtree
    kdtree = tree

def calculateNormals(indices, points, radius=2):
    """Estimate plane normals for given indices using PCA."""
    global kdtree
    if kdtree is None:  # Ensure k-d tree is initialized
        kdtree = cKDTree(points)
    normals = []
    for idx in indices:
        neighbor_indices = kdtree.query_ball_point(points[idx], radius)
        if not neighbor_indices:  
            normals.append(np.array([0, 0, 0]))  # Default normal if no neighbors
            continue
        
        neighbors = points[neighbor_indices]
        mean = np.mean(neighbors, axis=0)
        norm = neighbors - mean
        cov = np.cov(norm.T)
        eig_val, eig_vec = np.linalg.eigh(cov)
        sorted_idx = np.argsort(eig_val)[::-1]
        eig_vec = eig_vec[:, sorted_idx]
        plane_direction = eig_vec[:, 2]  # Smallest eigenvector is normal
        normals.append(plane_direction)
    return np.array(normals)

def alignNormalsGlobally(normals, points, radius=2):
    """Ensure all normals are consistently oriented across the entire point cloud."""
    kdtree = cKDTree(points)
    visited = np.zeros(len(normals), dtype=bool)

    queue = [0]  # Start with the first point as the reference
    visited[0] = True

    while queue:
        idx = queue.pop(0)
        reference_normal = normals[idx]

        # Find neighboring points
        neighbor_indices = kdtree.query_ball_point(points[idx], radius)

        for neighbor_idx in neighbor_indices:
            if visited[neighbor_idx]:
                continue  # Skip already visited neighbors

            # Flip neighbor normal if it points in the opposite direction
            if np.dot(normals[neighbor_idx], reference_normal) < 0:
                normals[neighbor_idx] *= -1

            visited[neighbor_idx] = True
            queue.append(neighbor_idx)  # Add to queue for further checking

    return normals

def calculateNormalStandardDeviation(indices, points, normals, radius=2):
    """Compute standard deviation of normal directions for given indices."""
    standard_deviations = []

    for idx in indices:
        neighbor_indices = kdtree.query_ball_point(points[idx], radius)
        if not neighbor_indices:
            standard_deviations.append(0)  
            continue

        neighbor_normals = normals[neighbor_indices]
        #reference_normal = neighbor_normals[0]
        #dot_products = np.dot(neighbor_normals, reference_normal)
        #aligned_normals = neighbor_normals * np.sign(dot_products)[:, np.newaxis]

        std_dev = np.std(neighbor_normals, axis=0)
        variation_measure = np.sum(std_dev)
        standard_deviations.append(variation_measure)

    return np.array(standard_deviations)

def calculatePointwiseNormalVariance(pcd, radius=2, num_chunks=16, num_workers=4, verbose=False):
    """
    Compute normal variation for a given point cloud.
    
    Args:
        pcd (open3d.geometry.PointCloud): Input point cloud.
        radius (float): Neighborhood search radius.
        num_chunks (int): Number of chunks for parallel processing.
        num_workers (int): Number of worker processes.

    Returns:
        np.ndarray: Normalized variation values for each point.
    """
    myarray = np.asarray(pcd.points)
    indices = np.arange(len(myarray))
    kdtree = cKDTree(myarray)  # Build k-d tree
    chunk_size = len(myarray) // num_chunks
    chunked_indices = [indices[i:i + chunk_size] for i in range(0, len(indices), chunk_size)]

    if verbose: 
        print(f"Calculating pointwise PCA using {num_workers} workers and {num_chunks} chunks. This may take a while... (approx. 1 minute per 1M points)")
    # First pass: Compute normals
    start_time_first_pca = time.time()
    with multiprocessing.Pool(processes=num_workers, initializer=initKdtree, initargs=(kdtree,)) as pool:
        normals_chunks = pool.starmap(calculateNormals, [(chunk, myarray, radius) for chunk in chunked_indices])
    first_pca_duration = time.time() - start_time_first_pca
    if verbose:
        print(f"PCA calculation time: {first_pca_duration:.2f} seconds")

    all_normals = np.vstack(normals_chunks)  # Flatten normal array
    all_normals = alignNormalsGlobally(all_normals, myarray, radius=2)
    # Second pass: Compute standard deviation-based variation
    if verbose:
        print("Calculating pointwise standard deviation. This may take a while... (approx. 30 sec per 1M points)")
    start_time_standard_deviations = time.time()
    with multiprocessing.Pool(processes=num_workers, initializer=initKdtree, initargs=(kdtree,)) as pool:
        standard_deviation_chunks = pool.starmap(calculateNormalStandardDeviation, [(chunk, myarray, all_normals, radius) for chunk in chunked_indices])
    standard_deviations_duration = time.time() - start_time_standard_deviations
    if verbose:
        print(f"Standard deviation calculation: {standard_deviations_duration:.2f} seconds")

    standard_deviations = np.hstack(standard_deviation_chunks)  # Flatten

    # Normalize variation values
    max_variation = np.max(standard_deviations) if len(standard_deviations) > 0 else 1
    normalized_variation = standard_deviations / max_variation

    return all_normals, normalized_variation

In [3]:
point_cloud_location = "/home/chris/Code/PointClouds/data/ply/CircularVentilationGrateExtraCleanedFull.ply"
pcd = o3d.io.read_point_cloud(point_cloud_location)
all_normals, pointwise_variance = calculatePointwiseNormalVariance(pcd, radius=2)

In [4]:
class GaussMapVisualizer:
    def __init__(self, pcd, all_normals, standard_deviations, radius):
        self.pcd = pcd
        self.points = np.asarray(pcd.points)
        self.pcd.paint_uniform_color([0.6, 0.6, 0.6])
        self.kdtree = cKDTree(self.points)
        self.plane_directions = all_normals
        self.radius = radius
        self.standard_deviations = standard_deviations
        low, high = np.percentile(self.standard_deviations, [95, 100])
        self.core_indices = np.where((self.standard_deviations > low) & (self.standard_deviations <= high))[0]
        print(f"Found {len(self.core_indices)} core points out of {len(self.pcd.points)} total points.")
        self.reference_normal = None

        self.current_index = 0
        self.vis = o3d.visualization.VisualizerWithKeyCallback()
        self.vis.create_window("GaussMapVisualizer")

        self.vis.register_key_callback(262, self.next_neighborhood)
        self.vis.register_key_callback(264, self.show_random_core_point)
        self.vis.add_geometry(self.pcd)
        self.apply_variation_heatmap()
        self._update_neighborhood()

    def get_nearest_neighbor_directions(self, point, kdtree, pcd, plane_directions, radius=2):
        """ Get the directions of the k nearest neighbors to a given point. """
        idx = kdtree.query_ball_point(point, radius)
        nearest_points = np.asarray(pcd.points)[idx]
        nearest_directions = np.asarray(plane_directions)[idx]
        return idx, nearest_points, nearest_directions
    
    def create_normal_lines(self, neighbor_points, neighbor_directions, scale=0.2):
        """ Create line segments for the normal directions at each point. """
        line_set = o3d.geometry.LineSet()

        start_points = np.array(neighbor_points)
        end_points = start_points + scale * np.array(neighbor_directions)
        lines = [[start_points[i], end_points[i]] for i in range(len(neighbor_points))]
        line_set.points = o3d.utility.Vector3dVector(np.concatenate(lines, axis=0))
        line_indices = [[i, i + 1] for i in range(0, len(lines) * 2, 2)]
        line_set.lines = o3d.utility.Vector2iVector(line_indices)
        line_set.colors = o3d.utility.Vector3dVector(np.tile((0, 0, 1), (len(lines), 1)))
        return line_set
    
    def align_normals(self, reference_normal, neighbor_directions):
        aligned_normals = np.array(neighbor_directions)
        
        # Check dot product: If negative, flip the normal
        for i in range(len(aligned_normals)):
            if np.dot(reference_normal, aligned_normals[i]) < 0:
                aligned_normals[i] = -aligned_normals[i]

        return aligned_normals
    
    def calculate_normal_variation(self, normals):
        mu = np.mean(normals, axis=0)
        norm = normals - mu
        cov = np.cov(norm.T)
        eig_val, _ = np.linalg.eig(cov)
        sorted_idx = np.argsort(eig_val)[::-1]
        eig_val = eig_val[sorted_idx]
        eig_val_norm = eig_val / np.sum(eig_val)
        
        return mu, eig_val_norm, cov
    
    def update_gauss_map(self, normals):
        """ Update the Gauss Map visualization with the current neighborhood's normals. """
        normals = np.array(normals)
        normals /= np.linalg.norm(normals, axis=1, keepdims=True)  # Normalize to unit sphere

        # Create figure
        plt.figure("Gauss Map", figsize=(6, 6))
        plt.clf()  # Clear previous plot
        ax = plt.subplot(111, projection="3d")

        # Plot unit sphere
        u = np.linspace(0, 2 * np.pi, 30)
        v = np.linspace(0, np.pi, 20)
        x = np.outer(np.cos(u), np.sin(v))
        y = np.outer(np.sin(u), np.sin(v))
        z = np.outer(np.ones(np.size(u)), np.cos(v))
        ax.plot_surface(x, y, z, color="gray", alpha=0.3, edgecolor="none")  # Transparent sphere

        # Plot normal vectors
        for normal in normals:
            ax.quiver(0, 0, 0, normal[0], normal[1], normal[2], color="b", linewidth=1, arrow_length_ratio=0.1)

        ax.set_xlabel("X")
        ax.set_ylabel("Y")
        ax.set_zlabel("Z")
        ax.set_title(f"Gauss Map - Neighborhood {self.current_index}")

        plt.pause(0.1)  # Allow Matplotlib to update

    def show_random_core_point(self, vis):
        if len(self.core_indices) == 0:
            print("No core points found.")
            return

        # Pick a random core point
        self.current_index = random.choice(self.core_indices)

        # Visualize its neighborhood
        self._update_neighborhood()

        print(f"Showing core point {self.current_index}.")

    def _update_neighborhood(self):
        """ Update visualization for the current neighborhood. """
        # Get the currently selected point
        query_point = np.asarray(self.pcd.points)[self.current_index]

        # Get nearest neighbors
        idx, neighbor_points, neighbor_directions = self.get_nearest_neighbor_directions(query_point, self.kdtree, self.pcd, self.plane_directions, radius=self.radius)
        if self.current_index==0:
            self.reference_normal = neighbor_directions[0]

        #aligned_directions = self.align_normals(self.reference_normal, neighbor_directions)
        normal_mean, normal_variation, cov_after = self.calculate_normal_variation(neighbor_directions)

        if hasattr(self, "normal_lines"):
            self.vis.remove_geometry(self.normal_lines)
        self.normal_lines = self.create_normal_lines(neighbor_points, neighbor_directions, scale=2)
        self.vis.add_geometry(self.normal_lines)
        view_ctl = self.vis.get_view_control()
        lookat = query_point
        zoom = 0.080000000000000002
        front = [-0.024106890455448116,-0.57254772319971181,0.81951690799604338]
        up =  [0.014828165865396817,0.81946017828866602,0.57294427451208185]
        view_ctl.set_lookat(lookat)  # Set the point the camera is looking at
        view_ctl.set_up(up)      # Set the up direction of the camera
        view_ctl.set_front(front)  # Set the front direction of the camera
        view_ctl.set_zoom(zoom)          # Set the zoom factor of the camera

        #self.update_gauss_map(aligned_directions)

        self.vis.update_geometry(self.pcd)
        print(f"Neighborhood {self.current_index}/{len(self.pcd.points)} updated", flush=True)
        print(f'Aligned normals: {neighbor_directions}', flush=True)
        print(f"Normal mean: {normal_mean}, Normal variation: {normal_variation}", flush=True)
        print(f"Std Dev of Normals: {np.std(neighbor_directions, axis=0)}")
        print(f"Condition Number of Covariance: {np.linalg.cond(cov_after)}")
        print(15*"-", flush=True)

    def next_neighborhood(self, vis):
        """ Move to the next neighborhood when right arrow key is pressed. """
        self.current_index = (self.current_index + 500) % len(self.pcd.points)
        self._update_neighborhood()
    
    def normalize_variation_colors(self, variation_values):
        """ Normalize variation values to a colormap range. """
        min_val, max_val = np.percentile(variation_values, [2, 98])  # Robust normalization
        norm_variation = (variation_values - min_val) / (max_val - min_val + 1e-6)  # Normalize to [0,1]

        # Use a colormap (e.g., viridis) to visualize variation
        colors = cm.viridis(norm_variation)[:, :3]  # Extract RGB colors

        return colors

    def apply_variation_heatmap(self):
        """ Apply standard deviation-based variation as a heatmap to the point cloud. """
        colors = self.normalize_variation_colors(self.standard_deviations)

        self.pcd.colors = o3d.utility.Vector3dVector(colors)  # Assign colors to points

        # self.vis.update_geometry(self.pcd)  # Uncomment if using an Open3D visualizer
        print("Standard deviation heatmap applied!")


    def run(self):
        self.vis.run()  # Start the visualization loop (blocks until closed)
        self.vis.destroy_window()

In [6]:
viewer = GaussMapVisualizer(pcd, all_normals, pointwise_variance, 2)
viewer.run()

Found 105840 core points out of 2116800 total points.
Standard deviation heatmap applied!
Neighborhood 0/2116800 updated
Aligned normals: [[-3.90565909e-03 -9.98650489e-03  9.99942506e-01]
 [-3.66693712e-03 -1.12771203e-02  9.99929688e-01]
 [-3.35612349e-03 -3.23883072e-03  9.99989123e-01]
 [-3.41269632e-03 -5.92899940e-03  9.99976600e-01]
 [-3.29194969e-03 -4.47871933e-03  9.99984552e-01]
 [-2.88827029e-03 -4.60014818e-03  9.99985248e-01]
 [-5.04839428e-03 -7.02474194e-03  9.99962583e-01]
 [-5.53869324e-03 -7.01264522e-03  9.99960072e-01]
 [-3.96624761e-03 -5.74203309e-03  9.99975649e-01]
 [-4.77564253e-03 -8.83847020e-03  9.99949536e-01]
 [-5.44101443e-03 -6.93956078e-03  9.99961118e-01]
 [-4.12334481e-03 -4.76169651e-03  9.99980162e-01]
 [-3.71151111e-03 -5.86681110e-03  9.99975902e-01]
 [-4.13353904e-03 -5.44572753e-03  9.99976629e-01]
 [-4.83069875e-03 -4.02978721e-03  9.99980212e-01]
 [-3.97314756e-03 -5.31154477e-03  9.99978001e-01]
 [-4.84945662e-03 -3.02643674e-03  9.99983662e